In [1]:
import tensorflow as tf

In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.4.1'

In [4]:
image_gen = ImageDataGenerator(rescale=1./255)

In [5]:
from google.colab import drive

In [6]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
train_data = image_gen.flow_from_directory('/content/gdrive/My Drive/Image Processing/Train',target_size=(227,227))

Found 9966 images belonging to 4 classes.


In [8]:
test_data = image_gen.flow_from_directory('/content/gdrive/My Drive/Image Processing/Test',target_size=(227,227))

Found 2487 images belonging to 4 classes.


In [9]:
import tensorflow.keras.layers as layers

In [10]:
WBC_classifier = tf.keras.models.Sequential()

In [11]:
WBC_classifier.add(layers.Conv2D(96,(11,11),strides=(4,4),activation='relu',input_shape=(227,227,3)))

In [12]:
WBC_classifier.add(layers.BatchNormalization())

In [13]:
WBC_classifier.add(layers.MaxPool2D((3,3),strides=(2,2)))

In [14]:
WBC_classifier.add(layers.Conv2D(256,(5,5),padding='same',activation='relu'))

In [15]:
WBC_classifier.add(layers.BatchNormalization())

In [16]:
WBC_classifier.add(layers.MaxPool2D((3,3),strides=(2,2)))

In [17]:
WBC_classifier.add(layers.Conv2D(384,(3,3),padding='same',activation='relu'))
WBC_classifier.add(layers.BatchNormalization())
WBC_classifier.add(layers.Conv2D(384,(3,3),padding='same',activation='relu'))
WBC_classifier.add(layers.BatchNormalization())

In [18]:
WBC_classifier.add(layers.Conv2D(256,(3,3),padding='same',activation='relu'))
WBC_classifier.add(layers.BatchNormalization())

In [19]:
WBC_classifier.add(layers.MaxPool2D((3,3),strides=(2,2)))

In [20]:
WBC_classifier.add(layers.Flatten())

In [21]:
WBC_classifier.add(layers.Dense(4096,activation='relu'))
WBC_classifier.add(layers.Dropout(0.5))

In [22]:
WBC_classifier.add(layers.Dense(4096,activation='relu'))
WBC_classifier.add(layers.Dropout(0.5))

In [23]:
WBC_classifier.add(layers.Dense(4,activation='softmax'))

In [24]:
WBC_classifier.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [25]:
WBC_classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       8

In [26]:
WBC_classifier.fit_generator(train_data,validation_data=test_data,max_queue_size=120,workers=10,use_multiprocessing=True,epochs=50)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
312/312 [==============================] - 348s 1s/step - loss: 8.2285 - accuracy: 0.3158 - val_loss: 40.5956 - val_accuracy: 0.2493
Epoch 2/50
312/312 [==============================] - 59s 181ms/step - loss: 1.0763 - accuracy: 0.5579 - val_loss: 44.2714 - val_accuracy: 0.2493
Epoch 3/50
312/312 [==============================] - 63s 195ms/step - loss: 0.8657 - accuracy: 0.6538 - val_loss: 124.4601 - val_accuracy: 0.2493
Epoch 4/50
312/312 [==============================] - 62s 186ms/step - loss: 0.6755 - accuracy: 0.7402 - val_loss: 32.5776 - val_accuracy: 0.2501
Epoch 5/50
312/312 [==============================] - 64s 195ms/step - loss: 0.5483 - accuracy: 0.7921 - val_loss: 14.9402 - val_accuracy: 0.2437
Epoch 6/50
312/312 [==============================] - 63s 191ms/step - loss: 0.4747 - accuracy: 0.8312 - val_loss: 0.8394 - val_accuracy: 0.5875
Epoch 7/50
312/312 [==============================] - 62s 190ms/step - loss: 0.3841 - accuracy: 0.8644 - val_loss: 33.3301 - v

In [27]:
WBC_classifier.evaluate_generator(test_data,max_queue_size=120,workers=10,use_multiprocessing=True)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[3.0929811000823975, 0.6344993710517883]

In [28]:
WBC_classifier.save('WBC_classifier.h5')